# Zillow Bestimate
## Wrangle Notebook

In [1]:
# imports
import pandas as pd
import numpy as np
import os
import acquire

### Data Acquisition

In [2]:
df = acquire.get_zillow_data()

In [3]:
# what do we have to work with?
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28124 entries, 0 to 28123
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   parcel_id          28124 non-null  int64  
 1   bath               28124 non-null  float64
 2   bed                28124 non-null  float64
 3   sqft_calc          28077 non-null  float64
 4   fips               28124 non-null  float64
 5   yearbuilt          28053 non-null  float64
 6   taxvaluedollarcnt  28123 non-null  float64
 7   tax_amount         28123 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 1.9 MB


### Data Preparation

In [ ]:
# do we have any whitespace?
df = df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
# how many null values do I have?
df.isnull().sum()

In [ ]:
# let's drop them
df = df.dropna()

In [ ]:
df.info()

In [ ]:
# do we have any duplicates
df = df.drop_duplicates()
df.info()

In [ ]:
# let's add a column to show the age of the house, not just year built
df['age'] = 2017 - df['yearbuilt']

In [ ]:
df.info()

In [ ]:
df.tax_amount.unique()

In [ ]:
# let's convert floats to integers where possible
df['bed'] = df['bed'].astype(int)
df['sqft_calc'] = df['sqft_calc'].astype(int)
df['fips'] = df['fips'].astype(int)
df['yearbuilt'] = df['yearbuilt'].astype(int)
df['taxvaluedollarcnt'] = df['taxvaluedollarcnt'].astype(int)
df['age'] = df['age'].astype(int)

In [ ]:
df.info()

In [ ]:
# calculate upper and lower bounds for square_feet
sqft_25, sqft_75 = df['sqft_calc'].quantile([0.25, 0.75])
sqft_iqr = sqft_75 - sqft_25
limit_upper = sqft_75 + (1.5 * sqft_iqr)
limit_lower = sqft_25 - (1.5 * sqft_iqr)

In [ ]:
# drop everything outside 1.5 time IQR
df = df[df['sqft_calc'] > limit_lower]
df = df[df['sqft_calc'] < limit_upper]

In [ ]:
df.info()